<h1><center><font color='red'>Argumentation Project</font></center></h1>

<h2><font color='green'>Initialisation</font></h2>

In [10]:
# Importation cell
import pandas as pd
import typing as t
import json


<h2><font color='green'>Dataframe creation and observations</font></h2>

<h3><font color='orange'><center>arg_context_con.csv<center></font></h3>

In [6]:
df_arg_context_con = pd.read_csv('Data/arg_context_con.csv',sep=';')
df_arg_context_con.head()

,Unnamed: 0,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a88,a89,a90,a91,a92,a93,a94,a95,a96,a97
0,micro_k017_con,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,micro_b027_con,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,micro_k003_con,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,micro_k016_con,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,micro_b018_con,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3><font color='orange'><center>arg_context_glob.csv<center></font></h3>

In [7]:
df_arg_context_glob = pd.read_csv('Data/arg_context_glob.csv',sep=';')
df_arg_context_glob.head()

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a89,a90,a91,a92,a93,a94,a95,a96,a97,Unnamed: 98
0,1,1,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,NaN
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,NaN
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,1,1,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


<h3><font color='orange'><center>arg_context_pro.csv<center></font></h3>

In [8]:
df_arg_context_glob = pd.read_csv('Data/arg_context_pro.csv',sep=';')
df_arg_context_glob.head()

,Unnamed: 0,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a88,a89,a90,a91,a92,a93,a94,a95,a96,a97
0,micro_b033_pro,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,micro_b026_pro,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,micro_k002_pro,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,micro_b032_pro,1,0,0,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
4,micro_b024_pro,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3><font color='orange'><center>complete_context_withlabels.csv<center></font></h3>

In [9]:
df_arg_context_glob = pd.read_csv('Data/complete_context_withlabels.csv',sep=';')
df_arg_context_glob.head()

,Unnamed: 0,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a88,a89,a90,a91,a92,a93,a94,a95,a96,a97
0,micro_k017_con,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,micro_b033_pro,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,micro_b027_con,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,micro_k003_con,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,micro_b026_pro,1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3><font color='orange'><center>arg_attr_patterns.json<center></font></h3>

In [12]:
f = open('Data/arg_attr_patterns.json')
 
data = json.load(f)
 
for i in data:
    print(i)

f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
